In [1]:
import nltk
import numpy as np
import pandas as pd
import pickle
import pprint
import project_helper
import project_tests
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests

In [2]:
# base URL for the SEC EDGAR browser
HEADER = {'Host': 'www.sec.gov', 'Connection': 'close',
          'Accept': 'application/json, text/javascript, */*; q=0.01',
          'X-Requested-With': 'XMLHttpRequest',
          'User-Agent': 'ruizhuoj@andrew.cmu.edu'
         }
endpoint = r"https://www.sec.gov/cgi-bin/browse-edgar"

# cik_lookup = {
#     'AMZN': '0001018724', }
# # 'BMY': '0000014272',
# # 'CNP': '0001130310',
# # 'CVX': '0000093410',
# # 'FL': '0000850209',
# # 'FRT': '0000034903',
# # 'HON': '0000773840'}
# define our parameters dictionary
param_dict = {'action':'getcompany',
              'CIK':'0001018724',
              'type':'10-k',
              'dateb':'20220101',
              'owner':'exclude',
              'start':'95',
              'output':'',
              'count':'100'}

# request the url, and then parse the response.
response = requests.get(url = endpoint, params = param_dict, headers=HEADER)
soup = BeautifulSoup(response.content, 'html.parser')

# Let the user know it was successful.
print('Request Successful')
print(response.url)

Request Successful
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001018724&type=10-k&dateb=20220101&owner=exclude&start=95&output=&count=100


In [3]:
doc_table = soup.find_all('table', class_='tableFile2')

# define a base url that will be used for link building.
base_url_sec = r"https://www.sec.gov"

master_list = []

# loop through each row in the table.
for row in doc_table[0].find_all('tr'):

    # find all the columns
    cols = row.find_all('td')

    # if there are no columns move on to the next row.
    if len(cols) != 0:

        # grab the text
        filing_type = cols[0].text.strip()
        filing_date = cols[3].text.strip()
        filing_numb = cols[4].text.strip()

        # find the links
        filing_doc_href = cols[1].find('a', {'href':True, 'id':'documentsbutton'})
        filing_int_href = cols[1].find('a', {'href':True, 'id':'interactiveDataBtn'})
        filing_num_href = cols[4].find('a')

        # grab the the first href
        if filing_doc_href != None:
            filing_doc_link = base_url_sec + filing_doc_href['href']
        else:
            filing_doc_link = 'no link'
        # create and store data in the dictionary
        file_dict = {}
        file_dict['file_type'] = filing_type
        file_dict['file_date'] = filing_date
        file_dict['links'] = {}
        file_dict['links']['documents'] = filing_doc_link
        # let the user know it's working
        print('-'*100)
        print("Filing Type: " + filing_type)
        print("Filing Date: " + filing_date)
        print("Filing Number: " + filing_numb)
        print("Document Link: " + filing_doc_link)

        # append dictionary to master list
        master_list.append(file_dict)

In [5]:
print(master_list)

[]
